# Feedforward Neural Network from scratch for Text Classification

### Contents

1. Goal
2. Approach
3. Dataset

### 1. Goal

- The goal of this notebook is to develop a Feedforward Neural Netowork from scratch for text classification.



### 2. Approach

- Transform raw text data into input vectors


- Feedforward network with
    - Input: Embedding weight matrix
    - Interim: Hidden layer(S)
    - Output: softmax
    
    
- Stochastic Gradient Descent (SGD) algorithm with
    - Forward pass to compute intermediate outputs
    - Backward pass (Backpropagation) to compute gradients and update weights of the network
    - Dropout for regularisation
    - Perform hyperparater tuning
        - dropout rate
        - embedding size
        - Learning rate
    - Plot the learning process
        - training loss
        - validation loss
        
        
- Pre-trained word embeddings (GloVe)
    - Reinitialise network weights with pre-trained word embeddings from GloVe
    - Don't update weight during training (weight freezing, NO backpropagation)
    - Perform hyperparameter tuning
    - Plot the learning process
    
    
- More hidden layers


### 3. Dataset

#### AG News Corpus
- The data taken from [AG News Corpus](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html).
    
    - Create folder structure as `./data_topic` with below files:
        - `data_topic/train.csv`: contains 2,400 news articles, 800 for each class to be used for training.
        - `data_topic/dev.csv`: contains 150 news articles, 50 for each class to be used for hyperparameter selection and monitoring the training process.
        - `data_topic/test.csv`: contains 900 news articles, 300 for each class to be used for testing.

#### GloVe Embeddings

- The pre-trained GloVe embeddings trained on Common Crawl (840B tokens, 2.2M vocab, cased, 300d vectors, 2.03 GB download) is downloaded from [here](http://nlp.stanford.edu/data/glove.840B.300d.zip).
